# Рекомендательные системы. Рекомендации через поиск ближайших соседей

> На этом практическом занятии мы с вами сделаем следующее:
- Посмотри как работает FAISS.
- Построим простейший сервис для рекомендаций используя FAISS.

## FAISS

> Необходимо сперва установить faiss библиотеку. Инструкцию можно найти [здесь](https://github.com/facebookresearch/faiss/blob/master/INSTALL.md).
```conda install faiss-cpu -c pytorch # CPU version only```

In [ ]:
import numpy as np

In [ ]:
dim = 512  # рассмотрим произвольные векторы размерности 512
nb = 10000  # количество векторов в индексе
nq = 1 # количество векторов в выборке для поиска
np.random.seed(58) # DON't CHANGE THIS
vectors = np.random.random((nb, dim)).astype('float32')
query = np.random.random((nq, dim)).astype('float32')

In [ ]:
vectors.shape

In [ ]:
vectors

### IndexFlatL2

Создаем Flat индекс и добавляем векторы без обучения

In [ ]:
import faiss # предварительно необходимо установить FAISS см. выше

########################
# YOUR CODE HERE
########################
index = ...
index.add(...)
print(index.ntotal)

Проведем поиск по нашим векторам из query:

In [ ]:
%%time
########################
# YOUR CODE HERE
########################
D, I = ...

print(I)
print(D)

### Inverted File Index

Необходмио создать quantiser(IndexFlatL2), индекс (IndexIVFFlat), обучить индекс и добавить вектора в индекс.

In [ ]:
%%time
k = 10 # количество центроидов
########################
# YOUR CODE HERE
########################
quantiser = ...
index = ...
index.train(...)
index.add(...)

Необходимо произвести поиск по индексу нашего запроса (query).

In [ ]:
%%time
########################
# YOUR CODE HERE
########################
D, I = ...
print(I)
print(D)

## Применим FAISS для рекомендаций в нашей задаче

Построим простейший рекомендательный сервис.

In [ ]:
import faiss
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF
from flask import Flask, jsonify, request

# constants
RANDOM_STATE = 57
N_FACTOR = 20 # размерность эмбедингов
N_RESULT = 10 # сколько фильмов рекомендуем

In [ ]:
ratings = pd.read_csv("ml-latest-small/ratings.csv")
movies = pd.read_csv("ml-latest-small/movies.csv")

In [ ]:
users = sorted(numpy.unique(ratings['userId']))
movies = sorted(numpy.unique(ratings['movieId']))

In [ ]:
# for later use
user_id2i = {id: i for i, id in enumerate(users)}
movie_id2i = {id: i for i, id in enumerate(movies)}
movie_i2id = {i: id for i, id in enumerate(movies)}

In [ ]:
# make sparse matrix
rating_mat = coo_matrix(
    (ratings['rating'], (ratings['userId'].map(user_id2i), ratings['movieId'].map(movie_id2i)))
)

In [ ]:
rating_mat.todense()

In [ ]:
# decompose
model = NMF(n_components=N_FACTOR, init='random', random_state=RANDOM_STATE)
user_mat = model.fit_transform(rating_mat)
movie_mat = model.components_.T

> **NMF** = Non-negative Matrix Factorization. Можно применять метод чередующихся наименьших квадратов (ALS) для неотрицательного матричного разложения. Ключевая идея - искать поочередно то столбцы $p_t$, то столбцы $q_t$ при фиксированных остальных.

In [ ]:
# indexing
# movie_index = faiss.IndexFlatL2(N_FACTOR)

########################
# YOUR CODE HERE
########################
k = 100 # количество центроидов
# необходимо дописать методы
quantiser = ...
movie_index = ...
movie_index.train(...)
movie_index.add(...)

In [ ]:
# create app
app = Flask(__name__)

In [ ]:
# API endpoint
@app.route('/')
def recom_for_user():
    user_id = request.args.get('user_id', default = 1, type = int)
    user_i = user_id2i[user_id]
    
    ########################
    # YOUR CODE HERE
    ########################
    # необходимо определить вектор пользователя (пользовательский эмбединг) и найти ближайшие к этому вектору индексы из фильмов
    user_vec = ...
    scores, indices = ...
    
    
    movie_scores = zip(indices[0], scores[0])
    return jsonify(
        movies=[
            {
                "id": int(movie_i2id[i]),
                "score": float(s),
            }
            for i, s in movie_scores
        ],
    )

In [ ]:
app.run(host="0.0.0.0", port=5000)

> Note: use this link in your browser to acccess your server: http://0.0.0.0:5000/?user_id=128